In [7]:
# ----- Packages -----
import PyWGCNA
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import binom


In [17]:
# ----- Funciones -----
def binomial_test(observed, total_genes_reference, genes_in_category_reference, alpha=0.05):
    """
    Perform the binomial test to calculate the p-value for over-representation of a category in the uploaded gene list.

    Parameters:
        observed (int): The number of genes observed in the uploaded gene list that belong to the category of interest.
        total_genes_reference (int): The total number of genes in the reference set.
        genes_in_category_reference (int): The number of genes in the reference set that belong to the category of interest.
        alpha (float): The significance level (default is 0.05).

    Returns:
        p_value (float): The calculated p-value.
    """
    # Calculate the probability p(C) from the reference set
    p_C = genes_in_category_reference / total_genes_reference

    # Calculate the p-value using the binomial distribution
    p_value = 1 - binom.cdf(observed - 1, total_genes_reference, p_C)
    # If observed is less than or equal to p(C)*total_genes_reference, calculate the under-representation p-value
    if observed <= p_C * total_genes_reference:
        p_value = binom.cdf(observed, total_genes_reference, p_C)

    return p_value

def enriquecimiento_en_cluster(df, tot_cadena):
    cant_genes = df['Total de genes'].sum()
    for cluster in df.index:
        cant_cluster = df.loc[cluster, 'Total de genes']
        cant_cadena = df.loc[cluster, 'Genes predichos']
        deberia = (cant_cluster * tot_cadena) / cant_genes
        
        # Calculate the observed fold change
        fold_change = cant_cadena / deberia

        # Calculate the p-value
        p_value = binomial_test(cant_cadena, cant_genes, cant_cluster)
        df.loc[cluster, 'Fold change'] = fold_change
        df.loc[cluster, 'p-value'] = p_value

    return df


In [2]:
# Nombre del archivo de texto que contiene la lista
nombre_archivo = r"..\Archivos\Genes_predichos\interseccion_ds_todos.txt"

# Lista donde se almacenarán los elementos leídos del archivo
predichos = []

# Abre el archivo en modo de lectura
with open(nombre_archivo, 'r') as archivo:
    # Lee cada línea del archivo
    for linea in archivo:
        # Elimina el carácter de nueva línea al final de la línea y agrega el elemento a la lista
        predichos.append(linea.strip())

In [3]:
grafo = PyWGCNA.readWGCNA(r"..\Archivos\Grafo_co-expresion\grafo_sin_sacar_0s.p")


Reading 5xFAD WGCNA done!


In [14]:
genes_modulos = pd.DataFrame(grafo.datExpr.var)
modulos = genes_modulos['moduleColors'].unique()
modulos2 = {}
    #para sumar el gen a cada uno de las variables de modulos2
cont = 0
for genes_str in modulos:
        df = genes_modulos[genes_modulos['moduleColors'] == genes_str]
        df_genes = df.index.values.tolist()
        modulos2[genes_str] = df_genes
        cont +=1
modules = grafo.datExpr.var.moduleColors.unique().tolist()
module_colors = pd.DataFrame(genes_modulos['moduleColors'])
analisis_network = pd.DataFrame(data=0, index=['Total de genes', 'Genes predichos'] , columns = modules)
for modulo in modulos2:
        analisis_network.loc['Total de genes', modulo] += len(modulos2[modulo])
    
genes_cadena_presentes = len(predichos)
for gen_interes in predichos: 
        try:
            modulo = module_colors.loc[gen_interes, 'moduleColors']
            analisis_network.loc['Genes predichos', modulo] +=1

        except:
            print('El gen', gen_interes, 'no está en el grafo')
            genes_cadena_presentes -= 1


El gen WBGene00014454 no está en el grafo


In [15]:
analisis_network.T

,Total de genes,Genes predichos
lightcoral,1704,18
coral,718,24
snow,597,6
whitesmoke,960,13
silver,842,8
sienna,180,7
lightsalmon,655,1
darkseagreen,75,1
blanchedalmond,114,0
darkred,483,4


In [18]:
sin0_enric = enriquecimiento_en_cluster(analisis_network.T, genes_cadena_presentes)


In [19]:
sin0_enric

,Total de genes,Genes predichos,Fold change,p-value
lightcoral,1704,18,1.757249,0.000000e+00
coral,718,24,5.560544,0.000000e+00
snow,597,6,1.671889,8.930247e-251
whitesmoke,960,13,2.252696,0.000000e+00
silver,842,8,1.580551,0.000000e+00
sienna,180,7,6.469281,3.489064e-67
lightsalmon,655,1,0.253974,5.434315e-288
darkseagreen,75,1,2.218039,1.731520e-31
blanchedalmond,114,0,0.000000,2.105545e-50
darkred,483,4,1.377664,3.994463e-204


In [20]:
sin0_enric.to_csv(r'..\Archivos\Grafo_co-expresion\Enriquecimiento de genes predichos en el grafo.csv')